In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import asyncio
import aiohttp
import aiofiles
from Crypto.Cipher import AES
import os

def get_iframe_src(url):
    resp = requests.get(url)
    main_page = BeautifulSoup(resp.text, "html.parser")
    src = main_page.find("iframe").get("src")
    return src

def get_first_m3u8_url(url):
    resp = requests.get(url)
    obj = re.compile(r'var main = "(?P<m3u8_url>.*?)";',re.S)
    m3u8_url = obj.search(resp.text).group("m3u8_url")
    return m3u8_url

def download_m3u8_file(url,name):
    resp = requests.get(url)
    with open(name, mode="wb") as f:
        f.write(resp.content)


async def download_ts(url, name, session):
    async with session.get(url) as resp:
        async with aiofiles.open(f"video2/{name}", mode="wb") as f:
            await f.write(resp.content.read()) # 下载的内容写入到文件中


async def aio_download(up_url):
    tasks = []
    async with aiohttp.ClientSession() as session: # 提前准备session
        async with aiofiles.open("越狱第一季第一集_second_m3u8.txt", mode = "r", encoding="utf-8") as f:
            async for line in f:
                if line.startswith('#'):
                    continue
                # line就是xxxxx.ts
                line = line.strip()
                # 拼接真正的ts路径
                ts_url = up_url + line
                task = asyncio.create_task(download_ts(ts_url, line, session)) # 创建任务
                tasks.append(task)

            await asyncio.wait(tasks) # for循环之后直接await

def get_key(url):
    resp = requests.get(url)
    return resp.text

def dec_ts(name,key):
    aes = AES.new(key=key, IV=b"0000000000000000", mode=AES.MODE_CBC)
    async with aiofiles.open(f"video2/{name}",mode="rb") as f1,\
        aiofiles.open(f"video2/temp_{name}",mode="wb") as f2:
        
        bs = await f1.read() # 从源文件读取
        await f2.write(aes.decrypt(bs))


async def aio_dec(key):
    # 解密
    tasks = []
    async with aiofiles.open("越狱第一季第一集_second_m3u8.txt", mode = "r", encoding="utf-8") as f:
        async for line in f:
            if line.startswith('#'):
                continue
            line = line.strip()
            # 开始创建异步任务
            task = asyncio.create_task(dec_ts(line,key))
            tasks.append(task)

        await asyncio.wait(tasks)


def merge_ts():
    # mac: cat 1.ts 2.ts 3.ts > xxx.mp4
    # win: copy /b 1.ts + 2.ts + 3.ts xxx.mp4
    lst = []
    with open("越狱第一季第一集_second_m3u8.txt", mode = "r", encoding="utf-8") as f:
        for line in f:
            if line.startswith("#"):
                continue
            line = line.strip()
            lst.append(f"video2/temp_{line}")
        # mac
        s = " ".join(lst)
        os.system(f"cat {s} > movie.mp4")
        # 注意，这里Windows系统要注意视频位置是否在代码路径下，若不在可以通过os.chdir('path)更改工作路径


def main(url):
    # 1. 拿到主页面的页面源代码，找到iframe对应的url
    iframe_src = get_iframe_src(url)
    # 2. 拿到第一层m3u8文件的下载地址
    first_m3u8_url = get_first_m3u8_url(iframe_src)
    # 拿到iframe的域名
    iframe_domain = iframe_src.split("/share")[0]
    # 拼接出真正的m3u8下载路径
    first_m3u8_url = iframe_domain + first_m3u8_url
    # 3.1 下载第一层m3u8文件
    download_m3u8_file(first_m3u8_url,"越狱第一季第一集_first_m3u8.txt")
    # 3.2 下载第二层m3u8文件
    with open("越狱第一季第一集_m3u8.txt", mode="r",encoding="utf-8") as f:
        for line in f:
            if line.startswith("#"):
                continue
            else:
                line = line.strip() # 去掉空白和换行符 hls/index.m3u8
                # 准备拼接第二层m3u8的下载路径
                second_m3u8_url = first_m3u8_url.split("index")[0] + line
                download_m3u8_file(second_m3u8_url,"越狱第一季第一集_second_m3u8.txt")

    # 4.下载视频
    second_m3u8_url_up = second_m3u8_url.replace("index.m3u8","")
    # 异步协程
    asyncio.run(aiodownload(second_m3u8_url_up))

    # 5.1 拿到密钥
    key_url = second_m3u8_url_up + "key.key" # 偷懒写法，正常需要去m3u8文件里去找
    key = get_key(key_url)
    # 5.2 解密
    asyncio.run(aio_dec(key))

    # 6.合并ts文件为MP4文件
    merge_ts()

if __name__ == "__main__":
    url = "https://www.91kanju.com"
    main(url)
    # 简单的问题复杂化，复杂的问题简单化